In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import clickhouse_connect
import os
from datetime import date, datetime

os.environ['NO_PROXY'] = '10.80.131.73'
connection = clickhouse_connect.get_client(host = "10.80.131.73", port = 8123, username = 'test', password = 'secretPassword321!', database = 'test')
path = 'Projects/XML_to_DB/CVE_SQL/CreateTable.sql'

allowed_val = ["Windows 10", 'Windows Server 2008','Windows 8', 'Windows 7', 'Windows Server 2012', 'Windows 11', 'Windows Server 2016', 'Windows Server 2019', 'Windows Server 2022', 'Windows Server']

doc_month_array = {8 :"Aug",9: "Sep",10: 'Oct',11: "Nov",12: "Dec",1: "Jan",2: 'Feb',3: "Mar",4: "Apr",5: "May",6: "Jun",7: "Jul"}



In [2]:
msrc = connection.query_df("""Select distinct product_names, doc_xml_date, doc_initial_relise_date,  kb, cve from msrc_data where cve not like 'ADV%'""")
print("Got msrc")
osquery_patches = connection.query_df('select distinct hostname, hotfix_id, installed_on from dwh_osquery_software_patches where toMonth(timestamp) >= toMonth(now()) -1 ')
print("Got osquery_patches")
osquery_version = connection.query_df('Select distinct * from dict_osquery_system_os_version where toMonth(timestamp) >= toMonth(now()) -1')
print("Got osquery_version")

Got msrc
Got osquery_patches
Got osquery_version


In [3]:

osquery_patches['hotfix_id'] = osquery_patches['hotfix_id'].str.lstrip('KB').astype(int)
osquery_patches = osquery_patches.loc[osquery_patches.groupby('hostname')['hotfix_id'].idxmax()]

msrc['kb'] = msrc['kb'].astype(int)

msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows 10' if x.startswith('Windows 10') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2008' if x.startswith('Windows Server 2008') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows 7' if x.startswith('Windows 7') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows 8' if x.startswith('Windows 8') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2012' if x.startswith('Windows Server 2012') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows 11' if x.startswith('Windows 11') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2016' if x.startswith('Windows Server 2016') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2019' if x.startswith('Windows Server 2019') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server 2022' if x.startswith('Windows Server 2022') else x)
msrc['product_names'] = msrc['product_names'].apply(lambda x: 'Windows Server' if x.startswith('Windows Server, version') else x)

osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows 10' if "Windows 10" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows Server 2019' if "Windows Server 2019" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows Server 2012' if "Windows Server 2012" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows Server 2016' if "Windows Server 2016" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows Server 2008' if "Windows Server 2008" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows 7' if "Windows 7" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows 11' if "Windows 11" in x else x)
osquery_version['os_name'] = osquery_version['os_name'].apply(lambda x:'Windows 8' if "Windows 8" in x else x)

msrc = msrc[msrc['product_names'].isin(allowed_val)]

msrc = msrc.sort_values(by = ['product_names', 'kb'])
print(osquery_patches)

              hostname  hotfix_id installed_on
46416        1c-app-05    5012647    4/28/2022
66116        1c-app-06    5044277   10/18/2024
66390        1c-app-11    5044277   10/18/2024
65899        1c-app-12    5044277   10/18/2024
66071        1c-app-13    5044277   10/18/2024
...                ...        ...          ...
62648     xa-vdi-sap01    5043124    10/4/2024
62493  xa-vdi-taxcom01    5043124    10/4/2024
33347  xa-vdi-taxcom03    5041974     9/8/2024
9295   xa-vdi-taxcomdb    5041913     9/8/2024
60783         xa-wem01    5043126    10/4/2024

[2471 rows x 3 columns]


In [4]:
res = (msrc.groupby('product_names')
       .apply(lambda x: sorted(x[['kb', 'doc_initial_relise_date']].drop_duplicates().assign(doc_initial_relise_date = x['doc_initial_relise_date'].dt.strftime('%Y-%m-%d')).values.tolist(), key = lambda y: y[1]))
       .reset_index(name = "kb_arr"))
print(len(res['kb_arr'][0]))
def rem_kb(arr):
    un_kb = {}
    for kb, time in arr:
        if kb not in un_kb or time > un_kb[kb]:
            un_kb[kb] = time
    return sorted([[kb, un_kb[kb]] for kb in un_kb], key= lambda x: x[0])

res['kb_arr'] = res['kb_arr'].apply(rem_kb)

msrc = msrc.sort_values(by = 'kb')
print(res)

418
         product_names                                             kb_arr
0           Windows 10  [[4528760, 2020-01-14], [4532691, 2020-02-11],...
1           Windows 11  [[5006674, 2021-10-12], [5007215, 2021-11-09],...
2            Windows 7  [[4534310, 2020-01-14], [4534314, 2020-01-14],...
3            Windows 8  [[4534297, 2020-01-14], [4534309, 2020-01-14],...
4       Windows Server  [[4528760, 2020-01-14], [4532693, 2020-02-11],...
5  Windows Server 2008  [[4534303, 2020-01-14], [4534310, 2020-01-14],...
6  Windows Server 2012  [[4534283, 2020-01-14], [4534288, 2020-01-14],...
7  Windows Server 2016  [[4534271, 2020-01-14], [4537764, 2020-02-11],...
8  Windows Server 2019  [[4532691, 2020-02-11], [4534273, 2020-01-14],...
9  Windows Server 2022  [[5005575, 2021-09-14], [5006699, 2021-10-12],...


In [5]:

osquery_version = pd.merge(osquery_version, res, left_on = 'os_name', right_on = 'product_names', how = 'left')
osquery_version = pd.merge(osquery_version, osquery_patches, on = 'hostname', how = 'left')

def trim_array(row):
    arr = row['kb_arr']
    kb = row['hotfix_id']

    
    if not isinstance(arr, list) : return []
    last_month = datetime.strptime(arr[-1][1], '%Y-%m-%d').date().month
    last_year = datetime.strptime(arr[-1][1], '%Y-%m-%d').date().year
    for i in range(len(arr)): 
        if arr[i][0] > kb and datetime.strptime(arr[i][1], '%Y-%m-%d').date().month == last_month and datetime.strptime(arr[i][1], '%Y-%m-%d').date().year == last_year: 
            return arr[i:]
   
    return []
            

osquery_version['kb_arr'] = osquery_version.apply(trim_array, axis = 1)


def get_cve(row):
    if row[ 'kb_arr'] == []: return []
        
    return msrc[(msrc['kb'] > row['hotfix_id']) & (msrc['product_names'] == row['os_name'])]['cve'].unique()
    
osquery_version['cve'] = osquery_version.apply(get_cve, axis = 1)

msrc_date = msrc[['kb', 'doc_xml_date']].drop_duplicates()
msrc_date['doc_xml_date'] = pd.to_datetime(msrc_date['doc_xml_date'], format = '%Y-%b') 
msrc_date.sort_values(by = 'doc_xml_date', ascending = False, inplace = True)
msrc_date = msrc_date.drop_duplicates(subset= ['kb'])
msrc_date['doc_xml_date'] = msrc_date['doc_xml_date'].dt.strftime('%Y-%b')
osquery_version = osquery_version.merge(msrc_date, left_on = 'hotfix_id', right_on = 'kb', how = 'left')
osquery_version = osquery_version.explode('cve')

In [6]:

with open('CreateTable.sql', 'r') as f:
    query = f.read()
    f.close()
query = query.split(';')
for i in query:
    connection.query(i)

In [7]:
osquery_version['hotfix_id'] = osquery_version['hotfix_id'].astype(str)
osquery_version['doc_xml_date'] = osquery_version['doc_xml_date'].astype(str)
osquery_version['cve'] = osquery_version['cve'].astype(str)
osquery_version['kb_arr'] = osquery_version['kb_arr'].apply(lambda x: [[str(sublist[0]), sublist[1]] if len(sublist)>1 else sublist for sublist in x])
osquery_version['hotfix_id'] = osquery_version['hotfix_id'].apply(lambda x: x[:-2] if len(x) > 0  and x!= 'nan' else '')
osquery_version['doc_xml_date'] = osquery_version['doc_xml_date'].apply(lambda x: x if x!= 'nan' else '')
osquery_version['cve'] = osquery_version['cve'].apply(lambda x: x if x!= 'nan' else '')

osquery_version['hostname'].replace([None], '', inplace = True)
osquery_version['codename'].replace([None], '', inplace = True)
osquery_version['version'].replace([None], '', inplace = True)
osquery_version['hotfix_id'].replace([None], '', inplace = True)
osquery_version['doc_xml_date'].replace([None], '', inplace = True)
osquery_version['install_date'].replace([None], '', inplace = True)
osquery_version['kb_arr'].replace([None], '', inplace = True)
osquery_version['cve'].replace([None], '', inplace = True)


res = osquery_version[['hostname', 'codename', 'version', 'hotfix_id', 'doc_xml_date', 'install_date', 'kb_arr', 'cve']]
# res[['hostname', 'codename', 'version', 'hotfix_id', 'doc_xml_date', 'install_date']] = res[['hostname', 'codename', 'version', 'hotfix_id', 'doc_xml_date', 'install_date']].astype(str)

# res.fillna('', inplace = True)


print(res.dtypes)

hostname        string[python]
codename        string[python]
version         string[python]
hotfix_id               object
doc_xml_date            object
install_date    string[python]
kb_arr                  object
cve                     object
dtype: object


In [8]:
print(res)

           hostname                                  codename     version  \
0     w64c901deb5d8  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
0     w64c901deb5d8  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
0     w64c901deb5d8  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
0     w64c901deb5d8  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
0     w64c901deb5d8  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
...             ...                                       ...         ...   
4106  n8c8caaeeb26b  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
4106  n8c8caaeeb26b  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
4106  n8c8caaeeb26b  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
4106  n8c8caaeeb26b  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   
4106  n8c8caaeeb26b  Майкрософт Windows 10 Корпоративная LTSC  10.0.17763   

     hotfix_id doc_xml_date install_date  \
0      5043126                1

In [9]:


connection.insert_df('host_rec_kb', res, column_names = ['hostname', 'os_name', 'os_version', 'last_kb', 'last_kb_date', 'last_update_date', 'rec_kb_arr', 'cve'])